<h1 style='text-align: center'>SQL Queries</h1>

## Getting Data From A SQL Database

### The Structure of a SQL Query

<img src='images/sql_statement.jpg'/>

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_matts
FROM student_courses
WHERE student_name = "Matt"
GROUP BY 1
HAVING COUNT(student_name) >= 3;
```

In [1]:
import sqlite3

conn = sqlite3.connect('tutorial.db')
c = conn.cursor()

In [2]:
columns = [x[0] for x in c.execute('select * from students').description]

In [3]:
columns

['name',
 'birthdate',
 'siblings',
 'birth_place',
 'years_in_nyc',
 'favorite_food']

In [4]:
import pandas as pd

In [6]:
pd.read_sql_query("""SELECT * FROM students""", conn)

,name,birthdate,siblings,birth_place,years_in_nyc,favorite_food
0,Sean Abu Wilson,02/06,2,"Birmingham, AL",7.60,guacamole
1,David Miller,06/06,2,"New York, NY",25.75,pizza
2,Abhijeet Kamble,05/07,0,Wardha India,0.50,Biriyani
3,Samantha Jackson,08/04,2,"Newport, RI",12.50,chocolate chip cookies
4,Anmol Srivats,11/19,0,"Bangalore, India",0.05,Sushi
5,Ran Tokman,03/19,2,"Haifa, Israel",8.00,cheesecake
6,Amy Li,12/29,1,"New York, NY",18.00,dumplings
7,Florencia Leoni,06/07,4,"Caracas, Venezuela",1.20,steak
8,Austin Krause,10/22,1,"Lansing, MI",0.17,Everything Bagels
9,Natalie Overchuk,01/02,2,"Kiev, Ukraine",0.10,pasta


### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


To run as DataFrame, pd.read_sql_query("""SELECT... FROM...""", conn).

1. What are the names of all of the students.

In [7]:
c.execute('''
SELECT name
FROM students
''').fetchall()

[('Sean Abu Wilson',),
 ('David Miller',),
 ('Abhijeet Kamble',),
 ('Samantha Jackson',),
 ('Anmol Srivats',),
 ('Ran Tokman',),
 ('Amy Li',),
 ('Florencia Leoni',),
 ('Austin Krause',),
 ('Natalie Overchuk',),
 ('Akshay Sharma',),
 ('Mohamad Eldebek',),
 ('Adam Dick',),
 ('Menachi Korn',),
 ('Miguel Peña',),
 ('Atiar Rahman',),
 ('Fhel Dimaano',),
 ('Maks Pazuniak',),
 ('Alex Mitrani',),
 ('Yish Lim',),
 ('Nicole Roach',),
 ('Omer Hakim',)]

2. Which student has the most siblings?

> This is great place to use a subquery. Encourage students who are initially struggling with a question along the lines of "How could you select the largest number of siblings that anyone has in the group?" From there, you can further push students with a hint if needed: "How can you now make a selection using the result of this, [embedded as a subquery]?"

In [20]:
c.execute("""
SELECT
    name,
    siblings
FROM students
WHERE siblings = (SELECT max(siblings)
                    FROM students)
""").fetchall()

[('Florencia Leoni', 4),
 ('Mohamad Eldebek', 4),
 ('Menachi Korn', 4),
 ('Miguel Peña', 4)]

3. How many students are only children?

In [28]:
c.execute("""
SELECT
    count(name),
    siblings
FROM students
WHERE siblings == 0
""").fetchone()

(3, 0)

4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
    

In [13]:
c.execute('''
SELECT
    name,
    years_in_nyc
FROM students
ORDER BY years_in_nyc
LIMIT 3
''').fetchall()

[('Anmol Srivats', 0.05), ('Natalie Overchuk', 0.1), ('Austin Krause', 0.17)]

5. How many students are native New Yorkers?

In [29]:
c.execute('''
SELECT count(name)
FROM students
WHERE birth_place LIKE "%NY"
''').fetchall()
#WHERE birth_place == "New York, NY" only returns those born in Manhattan

[(6,)]

6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [15]:
c.execute("""
SELECT
    favorite_food,
    count(favorite_food)
FROM students
GROUP BY favorite_food
HAVING count(favorite_food) > 1
""").fetchall()

[('pizza', 2), ('steak', 2)]

Which two students have the same favorite food?

In [38]:
c.execute("""
SELECT
    s1.name,
    favorite_food
FROM students s1
JOIN students s2
USING(favorite_food)
WHERE s1.name != s2.name
ORDER BY favorite_food
""").fetchall()

[('David Miller', 'pizza'),
 ('Akshay Sharma', 'pizza'),
 ('Florencia Leoni', 'steak'),
 ('Nicole Roach', 'steak')]

In [39]:
#Above command without join
c.execute("""
SELECT
    name,
    favorite_food
FROM students s1
WHERE favorite_food IN (SELECT favorite_food
                        FROM students
                        GROUP BY favorite_food
                        HAVING count(favorite_food) > 1)
ORDER BY favorite_food
""").fetchall()

[('David Miller', 'pizza'),
 ('Akshay Sharma', 'pizza'),
 ('Florencia Leoni', 'steak'),
 ('Nicole Roach', 'steak')]

## More Questions

What are the favorite foods of this classroom?

In [48]:
c.execute("""
SELECT
    favorite_food,
    count(favorite_food) as studentCount
FROM students
GROUP BY favorite_food
ORDER BY studentCount DESC
""").fetchall()

[('pizza', 2),
 ('steak', 2),
 ('Avocado', 1),
 ('Biriyani', 1),
 ('Everything Bagels', 1),
 ('Fusilli Sorrentina', 1),
 ('Kare Kare', 1),
 ('Reeses Puffs', 1),
 ('Sushi', 1),
 ('Tabouleh', 1),
 ('burgers', 1),
 ('cheesecake', 1),
 ('chocolate chip cookies', 1),
 ('dumplings', 1),
 ('falafel', 1),
 ('guacamole', 1),
 ('noodles', 1),
 ('pasta', 1),
 ('pho', 1),
 ('rice', 1)]

7. Which student was born closest to the cohort's graduation date?

In [43]:
#SQLite:
c.execute('''
SELECT name,
abs(julianday("2019-" || substr(birthdate, 1, 2) || "-" || substr(birthdate, 4, 2)) - julianday('2019-12-06')) AS days
FROM students
ORDER BY days
LIMIT 1
''').fetchall()

#MySQL:
#abs(date(concat(same as above without ||)) - date('2019-12-06'))

[('Alex Mitrani', 0.0)]